# Práctica 1

Autor: Blanca Cano Camarero  DGIIM  
Grupo: 2  
Fecha: Octubre 2021  


In [ ]:
# Configuración previa 
# modulos utilizados
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

# path carpeta con imágenes
path = './media'

# Ejercicio 1 
USANDO SOLO FUNCIONES BÁSICAS DE OPENCV, escribir funciones Python que implementen de forma eficiente el cálculo de los siguientes puntos. 

## Ejercicio 1A

Considere la función Gaussiana 1D de media 0 y desviación típica σ. Calcular las máscaras
discretas 1D de la función Gaussiana, la derivada de la Gaussiana y la segunda derivada de la
Gaussiana. La función implementada debe considerar que tanto el valor de σ como el tamaño
de la máscara son posibles entradas alternativas a la función. Represente en ejes cartesianos
las máscaras obtenidas como funciones 1D y compare sus formas con las máscaras dadas por
la función de OpenCV getDerivKernels para los mismos tamaños de máscara. Use los tamaños
5, 7 y 9.

### Solución 
La derivada primera de la Gaussiana se calcula como: 

Work!!
